In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 42 kB/s 
     |████████████████████████████████| 199 kB 52.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=9c1d8e2809b1c1356e4eb969476c34d64bd3f2991e554c46282ec16702c28f6d
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [2]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import * 

In [3]:
spark = SparkSession.builder.appName('Problem4').master('local[*]').getOrCreate()
sc = spark.sparkContext

In [4]:
file = sc.textFile("Points.txt")

In [5]:
file.take(10)

['6607,6538',
 '2911,1665',
 '8505,2858',
 '1717,7287',
 '6088,6936',
 '5320,6955',
 '8411,5550',
 '1971,9932',
 '2639,8086',
 '7672,4471']

In [6]:
def createPoint(text1):
    point1 = text1.split(",")
    x = int(point1[0])
    y = int(point1[1])
    return [x,y]

In [7]:
listOfPoints = file.map(createPoint)

In [8]:
listOfPoints.take(1)

[[6607, 6538]]

In [9]:
def createCell(p1):
  x = p1[0]
  y = p1[1]

  cell = ((int(((x - 1)/20)),  int(((y - 1)/20))) , 1)
  return cell

In [10]:
cellCoordinate = listOfPoints.map(createCell)

In [11]:
cellCoordinate.take(1)

[((330, 326), 1)]

In [12]:
cellCoordinateFinal = cellCoordinate.reduceByKey(lambda a, b: a + b)

In [13]:
cellCoordinateFinal.count()

250000

In [14]:
cellCoordinateFinal.take(5)

[((304, 346), 51),
 ((98, 496), 46),
 ((383, 223), 49),
 ((262, 320), 38),
 ((433, 389), 39)]

In [15]:
def getNeighbours(temp):
  x = temp[0][0]
  y = temp[0][1]
  selfPoints = temp[1]


  result = []

  for i in range(-1, 2):
    x1 = x + i
    for j in range(-1, 2):
      y1 = y + j
      if i==0 and j==0:
        continue

      if(x1>=0 and y1>=0 and x1<500 and y1<500):
        result.append((x1, y1))

  return (((x,y), selfPoints), result)

In [16]:
cellNeighbours = cellCoordinateFinal.map(getNeighbours)

In [17]:
cellNeighbours.count()

250000

In [18]:
cellNeighbours.take(1)

[(((304, 346), 51),
  [(303, 345),
   (303, 346),
   (303, 347),
   (304, 345),
   (304, 347),
   (305, 345),
   (305, 346),
   (305, 347)])]

In [20]:
coordinateOuput = cellCoordinateFinal.collect()

In [21]:
hashTableVal = {}
for i in coordinateOuput:
  key = i[0]
  val = i[1]
  hashTableVal[key] = val

In [22]:
cellIDAndRD = []
xyAndRD = []

for i in cellNeighbours.collect():

  selfValue = i[0][1]
  x = i[0][0][0]
  y = i[0][0][1]
  neighBourCount = 0
  n = len(i[1])

  for j in i[1]:
    neighBourCount += hashTableVal[(j[0], j[1])]

  RD = selfValue/(neighBourCount/n)

  cellID = x + y * 500 + 1
  xyAndRD.append([(x,y),RD])

  cellIDAndRD.append((cellID, RD))

In [23]:
cellIDAndRD

[(173305, 1.2142857142857142),
 (248099, 1.0791788856304985),
 (111884, 1.1563421828908556),
 (160263, 0.8328767123287671),
 (194934, 0.8501362397820164),
 (11265, 0.7115902964959568),
 (141872, 0.8986301369863013),
 (138714, 0.8772845953002611),
 (188554, 0.7167630057803468),
 (122251, 0.8914285714285715),
 (117277, 1.1398416886543536),
 (196834, 1.2237960339943343),
 (28423, 1.221556886227545),
 (152842, 1.318918918918919),
 (75952, 1.0756302521008403),
 (47441, 1.186046511627907),
 (9736, 1.008595988538682),
 (45205, 1.2087912087912087),
 (58271, 1.0279329608938548),
 (134650, 1.037037037037037),
 (18582, 1.0909090909090908),
 (20391, 1.2269938650306749),
 (57996, 1.0919220055710306),
 (231532, 0.8690807799442897),
 (33053, 1.0186335403726707),
 (31177, 0.9247311827956989),
 (158219, 0.9464788732394366),
 (140103, 1.0279329608938548),
 (39888, 1.0909090909090908),
 (129620, 0.9885057471264368),
 (223564, 1.1395348837209303),
 (200399, 0.9240121580547113),
 (99668, 1.0898550724637681

In [24]:
len(cellIDAndRD)

250000

In [25]:
rddFinal = sc.parallelize(cellIDAndRD)

#Step 2 Output (Top 50 Grid Cell w.r.t Relative Density Index)

In [26]:
rddFinal.sortBy(lambda x: x[1], ascending= False).take(50)

[(57253, 1.8909090909090909),
 (160955, 1.7926421404682273),
 (45314, 1.771986970684039),
 (55626, 1.7668711656441718),
 (85474, 1.75),
 (192689, 1.748502994011976),
 (133351, 1.7483443708609272),
 (187309, 1.7425742574257426),
 (56480, 1.7391304347826086),
 (144870, 1.7297297297297298),
 (233049, 1.7297297297297298),
 (45119, 1.7245508982035929),
 (86355, 1.7230769230769232),
 (115221, 1.7218543046357615),
 (148771, 1.7215189873417722),
 (198808, 1.7206703910614525),
 (186662, 1.7181208053691275),
 (132279, 1.7177914110429449),
 (71721, 1.7124600638977636),
 (207999, 1.7087378640776698),
 (117969, 1.7087378640776698),
 (213524, 1.7053291536050157),
 (22712, 1.7026239067055393),
 (195586, 1.696969696969697),
 (21242, 1.6908517350157728),
 (113482, 1.688073394495413),
 (43257, 1.6778115501519757),
 (70719, 1.6778115501519757),
 (154228, 1.6774193548387097),
 (20311, 1.6755162241887906),
 (112139, 1.6720257234726688),
 (142167, 1.6697819314641744),
 (34771, 1.6676737160120847),
 (75570, 

In [27]:
rddFinalXY = sc.parallelize(xyAndRD)

In [28]:
topPoints = rddFinalXY.sortBy(lambda x: x[1], ascending= False).take(50)

In [29]:
hashTable2 = {}

for i in cellNeighbours.collect():
  key = i[0][0]
  val = i[1]
  hashTable2[key] = val

In [30]:
hashTable2

{(304, 346): [(303, 345),
  (303, 346),
  (303, 347),
  (304, 345),
  (304, 347),
  (305, 345),
  (305, 346),
  (305, 347)],
 (98, 496): [(97, 495),
  (97, 496),
  (97, 497),
  (98, 495),
  (98, 497),
  (99, 495),
  (99, 496),
  (99, 497)],
 (383, 223): [(382, 222),
  (382, 223),
  (382, 224),
  (383, 222),
  (383, 224),
  (384, 222),
  (384, 223),
  (384, 224)],
 (262, 320): [(261, 319),
  (261, 320),
  (261, 321),
  (262, 319),
  (262, 321),
  (263, 319),
  (263, 320),
  (263, 321)],
 (433, 389): [(432, 388),
  (432, 389),
  (432, 390),
  (433, 388),
  (433, 390),
  (434, 388),
  (434, 389),
  (434, 390)],
 (264, 22): [(263, 21),
  (263, 22),
  (263, 23),
  (264, 21),
  (264, 23),
  (265, 21),
  (265, 22),
  (265, 23)],
 (371, 283): [(370, 282),
  (370, 283),
  (370, 284),
  (371, 282),
  (371, 284),
  (372, 282),
  (372, 283),
  (372, 284)],
 (213, 277): [(212, 276),
  (212, 277),
  (212, 278),
  (213, 276),
  (213, 278),
  (214, 276),
  (214, 277),
  (214, 278)],
 (53, 377): [(52, 

In [33]:
finalList = []
for i in topPoints:
  point = i[0]
  nList = hashTable2[point]
  ID = point[0] + point[1] * 500 + 1

  list1 = []

  for j in nList:
    RD = rddFinalXY.lookup(j)
    ID2 = j[0] + j[1] * 500 + 1
    list1.append([ID2,RD]) 

  finalList.append([ID, list1.copy()])
  list1.clear()


#Step 3 Output (Relative Density of the Top 50 Grid Cell's Neighbours)

In [34]:
finalList

[[57253,
  [[56752, [0.6091370558375635]],
   [57252, [1.229050279329609]],
   [57752, [0.7924528301886793]],
   [56753, [0.7035175879396985]],
   [57753, [0.7536231884057971]],
   [56754, [0.9487870619946092]],
   [57254, [0.7741935483870968]],
   [57754, [0.9702970297029703]]]],
 [160955,
  [[160454, [0.8328767123287671]],
   [160954, [0.7661971830985915]],
   [161454, [0.9662921348314607]],
   [160455, [0.7314285714285714]],
   [161455, [0.9341317365269461]],
   [160456, [1.032967032967033]],
   [160956, [0.7017543859649122]],
   [161456, [0.8228571428571428]]]],
 [45314,
  [[44813, [0.7486631016042781]],
   [45313, [0.7452054794520548]],
   [45813, [0.7671232876712328]],
   [44814, [1.182608695652174]],
   [45814, [1.0144092219020173]],
   [44815, [0.7851458885941645]],
   [45315, [0.6863270777479893]],
   [45815, [0.8478260869565217]]]],
 [55626,
  [[55125, [0.989010989010989]],
   [55625, [0.9411764705882353]],
   [56125, [0.5848563968668408]],
   [55126, [0.900804289544236]],
  